In [11]:
%pip install kronoslabs

from kronoslabs import KronosLabs

# Initialize the client. DONT FORGET TO PLUG IN THE API KEY!!!
client = KronosLabs(api_key="kl_8de5d1021ab88d725db4426791bcd805cf189a467e0465488a0567c0647b64bb")

# Non-streaming chat completion
response = client.chat.completions.create(
    prompt="Hello, how are you?",
    model="hermes",
    temperature=0.7,
    is_stream=False
)

print(response.choices[0].message.content) # <- prints the response the model gives

# Streaming chat completion
stream = client.chat.completions.create(
    prompt="Tell me a story",
    model="hermes",
    temperature=0.7,
    is_stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="") # <-prints the data as its being made

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Hello! I'm Hermes, an AI assistant created by Nous Research. I don't have personal feelings, but I'm here and ready to help you with any questions or tasks you may have. How can I assist you today?
Sure, I can tell you a story. Let's see... How about a classic fairy tale?

Once upon a time, in a far-off kingdom, there lived a young princess named Elara. She was known throughout the land for her kindness, beauty, and intelligence. However, the kingdom was ruled by a cruel and selfish king who only cared about his own power and wealth.

Elara dreamed of one day becoming a just and compassionate ruler, but she knew that her father would never allow it. One day, a mysterious stranger arrived at the castle gates, claiming to be a prince from a neighboring kingdom. He had heard of Elara's virtues and wished to marry her and unite the two kingdoms.

T

In [18]:
import google.generativeai as genai
import textwrap

# ==================== YOUR ORIGINAL FUNCTION ====================

def generate_detailed_feedback(user_analysis, ref_analysis, user_impact, ref_impact, threshold=15):
    """Generate detailed, actionable feedback"""
    feedback = []
    feedback.append("\n" + "="*60)
    feedback.append("BADMINTON SMASH FORM ANALYSIS")
    feedback.append("="*60)
    
    # 1. Elbow Angle Analysis
    elbow_diff = user_impact['elbow_angle'] - ref_impact['elbow_angle']
    feedback.append(f"\n📐 ELBOW ANGLE AT IMPACT:")
    feedback.append(f"   Your angle: {user_impact['elbow_angle']:.1f}°")
    feedback.append(f"   Reference: {ref_impact['elbow_angle']:.1f}°")
    feedback.append(f"   Difference: {elbow_diff:+.1f}°")
    
    if abs(elbow_diff) > threshold:
        if elbow_diff > 0:
            feedback.append(f"   ⚠️  Your elbow is too straight. Bend your elbow more during preparation")
            feedback.append(f"       for explosive power generation.")
        else:
            feedback.append(f"   ⚠️  Your elbow is too bent. Extend more at contact for maximum reach")
            feedback.append(f"       and racket head speed.")
    else:
        feedback.append(f"   ✅ Excellent elbow angle! This is optimal for power generation.")
    
    # 2. Shoulder Angle Analysis
    shoulder_diff = user_impact['shoulder_angle'] - ref_impact['shoulder_angle']
    feedback.append(f"\n💪 SHOULDER ANGLE AT IMPACT:")
    feedback.append(f"   Your angle: {user_impact['shoulder_angle']:.1f}°")
    feedback.append(f"   Reference: {ref_impact['shoulder_angle']:.1f}°")
    feedback.append(f"   Difference: {shoulder_diff:+.1f}°")
    
    if abs(shoulder_diff) > threshold:
        if shoulder_diff > 0:
            feedback.append(f"   ⚠️  Your shoulder is over-rotated. This may cause loss of control.")
        else:
            feedback.append(f"   ⚠️  Not enough shoulder rotation. Rotate your torso more into the shot")
            feedback.append(f"       to generate more power from your core.")
    else:
        feedback.append(f"   ✅ Good shoulder rotation!")
    
    # 3. Contact Point Height
    height_diff = user_impact['wrist_height'] - ref_impact['wrist_height']
    feedback.append(f"\n🎯 CONTACT POINT HEIGHT:")
    feedback.append(f"   Your height: {user_impact['wrist_height']:.3f}")
    feedback.append(f"   Reference: {ref_impact['wrist_height']:.3f}")
    feedback.append(f"   Difference: {height_diff:+.3f}")
    
    if abs(height_diff) > 0.05:  # 5% difference in normalized coordinates
        if height_diff > 0:
            feedback.append(f"   ⚠️  Contact point is too low. Jump higher or time your swing earlier")
            feedback.append(f"       for a steeper downward angle.")
        else:
            feedback.append(f"   ⚠️  Contact point might be too high. Ensure you're hitting at the")
            feedback.append(f"       optimal height for maximum power and control.")
    else:
        feedback.append(f"   ✅ Excellent contact point height!")
    
    # 4. Arm Extension
    extension_diff = user_impact['arm_extension'] - ref_impact['arm_extension']
    feedback.append(f"\n🏹 ARM EXTENSION AT IMPACT:")
    feedback.append(f"   Your extension: {user_impact['arm_extension']:.3f}")
    feedback.append(f"   Reference: {ref_impact['arm_extension']:.3f}")
    feedback.append(f"   Difference: {extension_diff:+.3f}")
    
    if abs(extension_diff) > 0.05:
        if extension_diff < 0:
            feedback.append(f"   ⚠️  Reach higher! Full arm extension maximizes racket head speed.")
        else:
            feedback.append(f"   ⚠️  You're over-extending. Keep some bend for control and prevent injury.")
    else:
        feedback.append(f"   ✅ Good arm extension!")
    
    # 5. Overall Score
    score = 100
    if abs(elbow_diff) > threshold: score -= 20
    if abs(shoulder_diff) > threshold: score -= 20
    if abs(height_diff) > 0.05: score -= 20
    if abs(extension_diff) > 0.05: score -= 15
    
    feedback.append(f"\n⭐ OVERALL FORM SCORE: {score}/100")
    
    if score >= 90:
        feedback.append("   🏆 Excellent form! Your technique matches professional standards.")
    elif score >= 70:
        feedback.append("   👍 Good form with minor adjustments needed.")
    elif score >= 50:
        feedback.append("   📈 Decent form but several areas need improvement.")
    else:
        feedback.append("   💪 Keep practicing! Focus on the key areas mentioned above.")
    
    return "\n".join(feedback)

# ==================== NEW AI FUNCTION ====================

def get_ai_coaching_feedback(analysis_report):
    """
    Sends the analysis report to a generative AI to get expert coaching.
    """
    # --- PASTE YOUR FREE API KEY HERE ---
    GOOGLE_API_KEY = 'YOUR_API_KEY_HERE' 
    
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
    except Exception as e:
        return f"Error configuring AI. Please check your API key. {e}"
        
    model = genai.GenerativeModel('gemini-pro')
    
    prompt = f"""
    You are an expert, encouraging badminton coach. 
    A student has just received the following technical analysis of their smash form, comparing them to a professional reference. 
    
    Your task is to analyze the report and:
    1.  Acknowledge any areas marked with "✅" as strong points.
    2.  Focus on the areas marked with "⚠️".
    3.  For each "⚠️" area, explain *why* it's important for a powerful smash (e.g., "Not rotating your shoulder means...").
    4.  Provide 1-2 specific, actionable drills or focus points for *each* of the problem areas.
    5.  Conclude with an encouraging summary based on their overall score.

    Here is their analysis report:
    ---
    {analysis_report}
    ---

    Provide your coaching advice in simple, clear markdown:
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error generating AI feedback: {e}"

print("All functions defined successfully.")

All functions defined successfully.


In [19]:
# ==================== FEEDBACK GENERATION ====================

print("IMPACT POINT ANALYSIS")
print("="*60)
# Make sure user_impact and ref_impact are defined!
print(f"User impact at frame: {user_impact['frame']}")
print(f"Reference impact at frame: {ref_impact['frame']}")


# 1. Generate and print the original feedback
# This line will now work because Cell 1 was run first.
feedback = generate_detailed_feedback(user_analysis, best_ref_analysis, user_impact, ref_impact)
print(feedback)

# 2. Call the new AI function and print the enhanced feedback
print("\n" + "="*60)
print("🤖 AI COACHING ANALYSIS (POWERED BY GEMINI)")
print("="*60)

ai_advice = get_ai_coaching_feedback(textwrap.dedent(feedback))
print(ai_advice)

IMPACT POINT ANALYSIS


NameError: name 'user_impact' is not defined